<a href="https://colab.research.google.com/github/ranajollanar/Real-Time-Emotion-Recognition/blob/main/Realtime_face_emotion_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'facial-expression:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F31050%2F39603%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240213%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240213T142140Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6d1d1ef4fd5666d73cd9f01f623d15254bc6066587e16fd7d98f2b56d15e1940b873a9f038cf39b6a4472751163a90497294cb9a7c8fc8785a0c7da3c19477662328b6d493f2828374b19b0cfd439a92c71d18ead94e4c5b99cfc7ba5806e1e75d9a7c6cba1cbf0e1f52db4c48507ab340dbc0c50a5226d59ee4d9b8e28ea7e2634ceaa675d7487ccf3c4daf84121aa6f84c4f9ed1d79bbd617e9a8d5cb861048b68577710703696e33539c9cd39f500e7d3cee0892b4ec807a4afb0c3d0b9101e838a11434cd1de727d056d53ea1f301a58eff78d243f2519170d5ff771b8793e170d4169a8332fffd421da4e027e517b289f401bb49b4c30dd15db2f54bcfb,testimages:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F35382%2F47835%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240213%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240213T142140Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D804584358fd20d8be00dc706fd83cf6c0ec6acf54086214738446968b465a7dd03f8ffca7d33c08e65788004f2a939fec7900cd737ce139605e0cb8f373218d13bb91e715f3f441e1e7d2bb582fd03e6cbe521125215dce625997ab50317c9eb78d5ca2cd74ee390b0cacc65591b6b3f8ab46f188ba81a7e6a062f1a827ec2ba7ce069e86b5f2c97a68da5d4a8b2979a59c1b0eb85607117cc80e65a6e70ada12192c0a17850ead364e893e8835daf4916e4adba3ecd197980b68e6b067c1178909fc4668e3f3b3bd3abccf0cbaab8688eafa4af484a2d50ed8b7c2543b17316440cfbb88b7f0870f7047ffa0c67b32d5b3a0017dc01eeac0a055c7d3f9ad841,myimage:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F35907%2F52857%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240213%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240213T142140Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4bd1450e1661d22c1356d0871d2382bc59f44a0f82e83cf824af12297fde3d38ced951ca81d3085a751d843bbf45418c66f70bd697830f54ebf2c19d6ccd57c6c959194fa5419f162eedcc757a8f15fb2b0d8f4fdb249582dbda8bf2340c10af80f78c7de30e64bcbd9229b36589acb8200dd0a9313e5cf9e966c0710ac33b7f6159a5a6bcc310c0aade6de4d568fae69a1c0f971419656489b47f847615cd98b009fb6af3adf614534b254bae036c09aec0490701daff532b52bc7046ee5f7aa1f74ca4a3835d3de6c9c6a151a78a525b15faef88c72242c4f6f74711d8bb7347f6b5de8580af73a8cacc9b60e06adae6b502fa56235b94f2fba79a821327d2'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 202559978 bytes downloaded
Downloaded and uncompressed: facial-expression
[==================================================] 333776 bytes downloaded
Downloaded and uncompressed: testimages
[==================================================] 45356 bytes downloaded
Downloaded and uncompressed: myimage
Data source import complete.


In [ ]:
img_array = cv2.imread("Training_314.jpg")

In [ ]:
# get the data
filname = '../input/facial-expression/fer2013/fer2013.csv'
label_map = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
names=['emotion','pixels','usage']
df=pd.read_csv('../input/facial-expression/fer2013/fer2013.csv',names=names, na_filter=False)
im=df['pixels']
df.head(10)

,emotion,pixels,usage
0,emotion,pixels,Usage
1,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
2,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
3,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
4,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
5,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
6,2,55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...,Training
7,4,20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...,Training
8,3,77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...,Training
9,3,85 84 90 121 101 102 133 153 153 169 177 189 1...,Training
